# pyjoins

This notebook aims to compare a few join algorithms using simple in-memory index tables and disk I/O with python 2.7

For this purposes, we are using simple CSV-like files containing the data that will be used for our joins. For indexing, we are using the default python dictionary for our hash index and the OOBtree from the BTrees module for our btree index. Also, the data used our tests is generated by a generator script at the project [repository](https://github.com/pboueke/pyjoins).

We will be testing nested loop joins, merge-joins and hash-joins. We will be testing for files (ordered and unordered) indexed with both the hash index and the btree index.


### Starting by loading our dependencies

In [2]:
from BTrees.OOBTree import OOBTree